# Random Forest

In [21]:
import pandas as pd
import numpy as np
from taxipred.utils.constants import DATA_PATH
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv(DATA_PATH / "taxi_cleaned.csv")

df.sample(5)

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
535,28.15,2.0,2.06,0.61,0.18,17.02,22.2951,False,False,False,False,False,False,False,False
448,35.14,2.0,4.17,1.05,0.11,113.49,53.5509,False,True,False,True,False,True,False,False
320,25.14,2.0,3.13,1.15,0.19,43.04,40.2186,False,False,False,False,False,True,True,False
37,1.27,2.0,4.47,0.62,0.16,37.99,11.3358,False,True,False,False,False,False,False,False
307,35.50,2.0,3.21,1.91,0.35,111.09,109.8965,False,False,False,True,True,False,True,False


### 0. Divide into X and y

In [22]:
df_train = df.dropna(subset=["Trip_Price"])

X, y = df_train.drop("Trip_Price", axis="columns"), df_train["Trip_Price"]

X.head()

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
0,19.35,3.0,3.56,0.80,0.32,53.82,False,True,False,False,True,False,False,False
1,36.87,1.0,2.70,1.21,0.15,37.27,True,False,False,True,False,False,False,False
2,8.64,2.0,2.55,1.71,0.48,89.33,False,False,False,True,False,True,False,False
4,41.79,3.0,4.60,1.77,0.11,86.95,False,False,True,True,False,False,False,False
5,9.91,2.0,2.32,1.26,0.34,41.72,True,False,False,False,False,False,False,False


In [23]:
y.info()

<class 'pandas.core.series.Series'>
Index: 548 entries, 0 to 573
Series name: Trip_Price
Non-Null Count  Dtype  
--------------  -----  
548 non-null    float64
dtypes: float64(1)
memory usage: 8.6 KB


### 1. train|test split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
    )


### 3. Training

In [25]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

### 4. Predict on test data

In [26]:
y_pred = rf.predict(X_test)
y_pred

array([ 44.364226,  31.329304, 104.561637,  25.911119,  44.883365,
        51.991433,  49.640192,  34.871338,  32.039854,  38.706476,
       106.59141 ,  48.352623,  31.606037,  29.838013,  44.650371,
        80.880808,  41.392552,  67.193874,  19.518511,  53.742112,
        65.540708,  54.964799,  25.064033,  58.492516,  31.304884,
        58.156504,  56.999617,  35.657605,  41.218056,  27.056809,
        27.859935,  55.283132,  22.261592,  95.465024,  80.837703,
        61.84621 ,  59.671498,  39.746267,  52.854379,  11.739291,
        92.609667,  71.84079 ,  12.52877 ,  61.292432,  68.503581,
        78.114103,  32.318716,  67.667048,  39.507826,  26.501871,
        53.577146,  18.209598,  69.647182,  71.441907,  90.531354,
        29.249389,  11.240629,  49.605856,  40.288817,  17.110578,
        56.443581,  17.346758,  43.811772,  72.88896 ,  78.298371,
        56.667567,  65.560308,  85.373006,  56.726222,  42.973956,
        15.950247,  35.952616,  67.920678,  55.508833,  49.810

### 5. Evaluation

In [28]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mae, mse, rmse

(3.7405536022099444, 24.413747955645764, np.float64(4.941027014259866))

### RandomForestRegression gave us RMSE = 4.94

### Feature importance

In [ ]:
rf.feature_importances_

array([0.5006518 , 0.00442628, 0.01155167, 0.25616785, 0.07436565,
       0.14350783, 0.00133401, 0.00136354, 0.00089066, 0.00130905,
       0.00125563, 0.00127619, 0.00129002, 0.00060981])

In [33]:
feature_importance = pd.DataFrame([X.columns, rf.feature_importances_]).T
feature_importance.columns = ["feature", "importance"]
feature_importance = feature_importance.sort_values(by="importance", ascending=False)
feature_importance

,feature,importance
0,Trip_Distance_km,0.500652
3,Per_Km_Rate,0.256168
5,Trip_Duration_Minutes,0.143508
4,Per_Minute_Rate,0.074366
2,Base_Fare,0.011552
1,Passenger_Count,0.004426
7,Time_of_Day_Morning,0.001364
6,Time_of_Day_Evening,0.001334
9,Day_of_Week_Weekend,0.001309
12,Weather_Rain,0.00129


## XGBoost